# Анализ сайта gismeteo.ru

Для выполнения итогового задания в качестве источника данных метеосводок был выбран сервис Дневник сайта Gismeteo.ru, в котором можно бесплатно получить исторические данные о погоде, начиная с 1997 года. Необходимо получить данные с 2008 по 2018 года включительно (последний год необходим для прогнозирования навигации на сезон).

todo: добавить список коммерческих ресурсов, с которых можно получить исторические данные и прогнозы на долгий срок

In [1]:
START_YEAR = 2008
END_YEAR = 2018

![Внешний вид сервиса Дневник от gismeteo](images/gismeteo_example.png)

После исполнения скрипта на загрузку данных из сайта автоматизированной информационной системы государственного мониторинга водных объектов Российской Федерации (далее - "АИС ГМВО") **src\preparation\scrape_water_level.py**, в **data\raw\water_posts_data.json** будет находиться словарь **{uid: наименование поста наблюдения}**. Наименования представлены в виде: *название водного объекта* - *название населённого пункта*

## Получение списка населённых пунктов с постами наблюдения

### Проверка и установка рабочей директории, должен быть корень проекта

In [2]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [3]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка списка постов

In [4]:
from src.utils import *

if not is_data_exists(DATA_POSTS_RAW, is_raw=True):
    print('Данные с постами наблюдения АИС ГМВО не были загружены. Пожалуйста, '
          'выполните скрипт src\preparation\scrape_water_level.py'
          'и попробуйте ещё раз.')
else:
    file_posts = open_file(DATA_POSTS_RAW, is_raw=True)
    posts = json.loads(file_posts)
print(len(posts))
posts  # основная переменная, в которой будет храниться вся информация о гидрологических постах

28


{'09386': 'р.Подкаменная Тунгуска - пос.Чемдальск',
 '09387': 'р.Подкаменная Тунгуска - с.Ванавара',
 '09388': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
 '09389': 'р.Подкаменная Тунгуска - с.Байкит',
 '09390': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
 '09392': 'р.Чуня - пос.Стрелка Чуня',
 '09393': 'р.Чуня - пос.Муторай',
 '09396': 'руч.Миханский - пос.Вельмо 2-е',
 '09397': 'р.Тея - пгт Тея',
 '09499': 'р.Бурная - пос.Бурный',
 '09523': 'р.Подкаменная Тунгуска - пос.Суломай',
 '09560': 'р.Вельмо - пос.Светлана',
 '09568': 'р.Вельмо - пос.Вельмо 2-е',
 '09403': 'р.Нижняя Тунгуска - д.Верхнекарелина',
 '09404': 'р.Нижняя Тунгуска - с.Подволошино',
 '09405': 'р.Нижняя Тунгуска - с.Преображенка',
 '09406': 'р.Нижняя Тунгуска - с.Ербогачен',
 '09408': 'р.Нижняя Тунгуска - пос.Кислокан',
 '09410': 'р.Нижняя Тунгуска - пгт Тура',
 '09413': 'р.Нижняя Тунгуска - пос.Учами',
 '09415': 'р.Нижняя Тунгуска - факт.Большой Порог',
 '09416': 'р.Непа - с.Токма',
 '09417': 'р.Непа - д.Ика',
 '09419'

### Создание набора с очищенными названиями населённых пунктов

In [5]:
#test_name = next(iter(posts.values()))  # р.Подкаменная Тунгуска - пос.Чемдальск
#test_name = posts['09388']  # р.Подкаменная Тунгуска - факт.Усть-Камо
test_name = posts['09415']  # р.Нижняя Тунгуска - факт.Большой Порог
test_name

'р.Нижняя Тунгуска - факт.Большой Порог'

In [6]:
import re

'''
Между видом и названием населённого пункта есть или точка, или пробел, используем [ \.]{1}
Учтено наличие в названии пункта пробела и тире
В БД gismeteo названия некоторых пунктов отличаются от таковых из сайта АИС ГМВО
'''
def clear_name(input_str):
    result = input_str.replace('Вельмо 2-е', 'Вельмо')
    result = result.replace('Стрелка Чуня', 'Стрелка-Чуня')
    result = result.replace('Большой порог', 'Большой Порог')
    result = re.search(r" - \w+[ \.]{1}([\w -]+)", result).group(1)
    return result

clear_name(test_name)

'Большой Порог'

In [7]:
for uid, name in posts.items():
    posts[uid] = {'name': name, 'clean_name': clear_name(name)}

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск'},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара'},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит'},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка'},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня', 'clean_name': 'Стрелка-Чуня'},
 '09393': {'name': 'р.Чуня - пос.Муторай', 'clean_name': 'Муторай'},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е', 'clean_name': 'Вельмо'},
 '09397': {'name': 'р.Тея - пгт Тея', 'clean_name': 'Тея'},
 '09499': {'name': 'р.Бурная - пос.Бурный', 'clean_name': 'Бурный'},
 '09523': {'name': 'р.Подкаменная Тунгуска - пос.Суломай',
  'clean_name': 'Суломай'},
 '09560': {'name': 'р.Вельмо - пос.Светлана', 'clean_name': 'Светлана'},
 '09568': {'name': 'р.Вельмо - пос.Вельм

### Создание списка с уникальными населёнными пунктами

In [8]:
names = set([x['clean_name'] for x in posts.values()])
names

{'Байкит',
 'Большой Порог',
 'Бурный',
 'Ванавара',
 'Вельмо',
 'Верхнекарелина',
 'Ербогачен',
 'Ика',
 'Кербо',
 'Кислокан',
 'Кузьмовка',
 'Муторай',
 'Непа',
 'Подволошино',
 'Преображенка',
 'Светлана',
 'Стрелка-Чуня',
 'Суломай',
 'Тембенчи',
 'Тея',
 'Токма',
 'Тура',
 'Усть-Камо',
 'Учами',
 'Чемдальск'}

In [9]:
print(f'Число постов: {len(posts)}\nЧисло уникальных пунктов: {len(names)}')

Число постов: 28
Число уникальных пунктов: 25


## Проверка наличия исторических метео-данных

С получением метео-данных есть 2 проблемы:
1. Не все населённые пункты представлены в списке городов.
2. В некоторых пунтах за период с 2008 по 2018 не велись наблюдения метео-данных. При попытке их получить сайт показывает сообщение:
> Наблюдения метео-данных в данный период не велись.

![Сообщение об ошибке](images/gismeteo_error.png)

Необходимо обнаружить проблемные населённые пункты.

### Получение списка населённых пунктов с gismeteo

После открытия https://www.gismeteo.ru/diary/ и выбора страны Россия и области Красноярский край происходит 3 GET запроса:
![GET запросы](images/gismeteo_requests.png)

1. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/countries/?fr=sel  
Получение списка стран c ID и названиями.

2. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/districts/?country=156&fr=sel  
Получения списка областей c ID и названиями. В параметре country указан ID выбранной страны (156 - Россия).

3. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel  
Получения списка городов c ID и названиями. В параметре district указан ID выбранной области (301 - Красноярский край, 303 - Иркутская область).

Во всех запросах **63466572668a39754a9ddf4c8b3437b0** - это стандартный api-токен, который не зависит от IP-адреса пользователя. Запросы возвращают данные в формате xml.
Учтём, что Подкаменная Тунгуска и Нижняя Тунгуска протекают в Красноярском крае и Иркутской области России, следовательно нужно запрашивать данные о пунктах в обоих субъектах.

In [10]:
from src.utils import get_url
import xmltodict

def get_cities(district):
    file_name = f'gismeteo_cities-{district}.xml'
    if is_data_exists(file_name, is_raw=True):
        cities = open_file(file_name, is_raw=True)
    else:
        url = f'https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district={district}&fr=sel'
        r = get_url(url)
        cities = r.text
        write_data(file_name, data=cities, is_raw=True)
    cities = xmltodict.parse(cities)  # конвертация xml в dict
    # основные данные находятся в cities['document']['item']
    cities = cities['document']['item']
    return cities

cities = {}
target_districts = [301, 303]  # нужные области
cities_file_name = f'gismeteo_cities-{",".join([str(x) for x in target_districts])}.json'
if is_data_exists(cities_file_name, is_raw=True):
    cities = open_file(cities_file_name, is_raw=True)
    cities = json.loads(cities)
else:
    for district in target_districts:
        for city in get_cities(district):
            ''' Записи в xml выглядят таким образом:
            <item id="145181" n="Абалаково" country_id="156" country_name="Россия" 
                  district_id="303" district_name="Иркутская область" kind="T"/>
                Нужные атрибуты - это id и n (name, имя)
            '''
            cities[int(city['@id'])] = city['@n']
    write_data(cities_file_name, data=cities, is_raw=True)
    
len(cities)

2861

In [11]:
for city_id, name in cities.items():
    for uid, entry in posts.items():
        if entry['clean_name'] == name:
            posts[uid].update({'gismeteo_id': int(city_id)})

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка',
  'gismeteo_id': 158143},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня',
  'clean_name': 'Стрелка-Чуня',
  'gismeteo_id': 158151},
 '09393': {'name': 'р.Чуня - пос.Муторай',
  'clean_name': 'Муторай',
  'gismeteo_id': 158146},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е',
  'clean_name': 'Вельмо',
  'gismeteo_id': 157875},
 '09397': {'name': 'р.Тея - пгт Тея',
  'clean_name': 'Тея',
  'gismeteo_id': 157881},
 '09499': {'name': 'р.Бурная - пос.Бурный',
  'clean_name': 'Бурн

Посмотрим, сколько населённых пунктов их списка постов гидрологического контроля рек есть в базе gismeteo:

In [12]:
len(names), len(set([x['clean_name'] for x in posts.values() if 'gismeteo_id' in x.keys()]))

(25, 19)

Для 25 уникальных локаций пунктов наблюдений нашлось соответствие с 19 населёнными пунктами в Gismeteo.

### Реализация проверки наличия метео-данных

Получение исторических метео-данных происходит следующим образом:
1. Задаются все необходимые параметры - страна, область, город, месяц и год
2. Нажатие на кнопку Получить дневник выполняет перенаправление на url вида  
https://www.gismeteo.ru/diary/4036/2008/1/  
в котором:  
- **4036** - ID населённого пункта
- **2008** - год
- **1** - месяц без лидирующего нуля

Если данные о погоде за указанный период не были зафиксированы, то вместо дневника будет сообщение об отсутствии данных.

In [13]:
import os
from bs4 import BeautifulSoup
from htmlmin import minify

def get_gismeteo_table(city_id, year, month):
    file_name = os.path.join('gismeteo', str(city_id), f'{year}-{month:02d}.html')
    if is_data_exists(file_name, is_raw=True):
        weather = open_file(file_name, is_raw=True)
        soup = BeautifulSoup(weather, 'lxml')
        if soup.find(class_='empty_phrase'):
            return None
        return soup
    else:
        url = f'https://www.gismeteo.ru/diary/{city_id}/{year}/{month}/'
        r = get_url(url)
        weather = r.text
        soup = BeautifulSoup(weather, 'lxml')
        empty_phrase = soup.find(class_='empty_phrase')
        if empty_phrase:
            write_data(file_name, data=str(empty_phrase), is_raw=True)
            return None
        
        table = soup.find('table')
        # для формирования форматированных html-страниц (с отступами и переносами строк)
        # можно использовать table.prettify(), для минификации (максимально возможного
        # сокращения размера html без потери фукнционала) - сторонний модуль htmlmin
        write_data(file_name, data=minify(str(table)), is_raw=True)
        return table

def check_gismeteo(city_id, year):
    return get_gismeteo_table(city_id, year, 1) is not None

check_gismeteo(4036, 2008)

True

Проверяем, есть ли за первый (2008) и последний год (2018) метео-данные у выбранных городов. Если у города нет соответствия с БД gismeteo, на конечный год у него нет данных, то изменяем его gismeteo_id на -1 для дальнейшей его замены на корректный город. Если на начальный год нет данных - добавляем ключ fallback_id, который будет использоваться для получения данных вместо основного id.

In [14]:
def get_bad_cities(print_info=False):
    result = []
    for uid, city in posts.items():
        if 'gismeteo_id' not in city.keys() or city['gismeteo_id'] == -1:
            if print_info:
                print(f'- {city["clean_name"]} нет в БД gismeteo')
            posts[uid].update({'gismeteo_id': -1})
            result.append(city)
        elif not check_gismeteo(city["gismeteo_id"], END_YEAR):
            if print_info:
                print(f'@ {city["clean_name"]} без метео-данных на {END_YEAR} год, нужна замена')
            posts[uid].update({'gismeteo_id': -1})
            result.append(city)
        elif not check_gismeteo(city["gismeteo_id"], START_YEAR):
            if 'fallback_id' in city.keys() and city['fallback_id'] != -1 \
                and check_gismeteo(city["fallback_id"], START_YEAR):
                if print_info:
                    print(f'+ {city["clean_name"]} имеет корректную подмену')
                continue
            if print_info:
                print(f'* {city["clean_name"]} без метео-данных на {START_YEAR} год, нужна подмена')
            posts[uid].update({'fallback_id': -1})
            result.append(city)
        elif print_info:
            print(f'+ {city["clean_name"]} имеет все необходимые метео-данные')
    return result

bad_cities = get_bad_cities(print_info=True)
print(f'Количество постов наблюдения с проблемными населёнными пунктами: {len(bad_cities)} пунктах')
posts

* Чемдальск без метео-данных на 2008 год, нужна подмена
+ Ванавара имеет все необходимые метео-данные
- Усть-Камо нет в БД gismeteo
+ Байкит имеет все необходимые метео-данные
* Кузьмовка без метео-данных на 2008 год, нужна подмена
* Стрелка-Чуня без метео-данных на 2008 год, нужна подмена
* Муторай без метео-данных на 2008 год, нужна подмена
* Вельмо без метео-данных на 2008 год, нужна подмена
* Тея без метео-данных на 2008 год, нужна подмена
* Бурный без метео-данных на 2008 год, нужна подмена
* Суломай без метео-данных на 2008 год, нужна подмена
- Светлана нет в БД gismeteo
* Вельмо без метео-данных на 2008 год, нужна подмена
- Верхнекарелина нет в БД gismeteo
* Подволошино без метео-данных на 2008 год, нужна подмена
* Преображенка без метео-данных на 2008 год, нужна подмена
+ Ербогачен имеет все необходимые метео-данные
* Кислокан без метео-данных на 2008 год, нужна подмена
+ Тура имеет все необходимые метео-данные
* Учами без метео-данных на 2008 год, нужна подмена
- Большой Порог

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback_id': -1},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': -1},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка',
  'gismeteo_id': 158143,
  'fallback_id': -1},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня',
  'clean_name': 'Стрелка-Чуня',
  'gismeteo_id': 158151,
  'fallback_id': -1},
 '09393': {'name': 'р.Чуня - пос.Муторай',
  'clean_name': 'Муторай',
  'gismeteo_id': 158146,
  'fallback_id': -1},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е',
  'clean_name': 'Вельмо',
  'gismeteo_id': 157875,
  'fallback_id': -1},
 '09397': {'name': 'р.Тея 

### Замена населённых пунктов

Так как для некоторых населённых пунктов не нашлось соответствие с списоком gismeteo, а также на начало 2008 года не было метео-сводок по 23 постам наблюдения, то на отсутствующие периоды времени для полноты исходных данных необходимо определить им замену. Оптимальнее всего будет вместо отсутствующих пунктов использовать их административные центры, т.к. географически они близко расположены друг к другу, а также по центрам вероятнее всего будут исторические погодные данные. 

Для определения административных центров будет использован сайт Википедии, где на страницах с населёнными пунктами указан муниципальный район, к которому он принадлежит, а на странице с этим районом - административный центр.

In [15]:
def get_wiki_page(page_name):
    file_name = os.path.join('wiki', f'{page_name}.html')
    if is_data_exists(file_name, is_raw=True):
        wiki = open_file(file_name, is_raw=True)
        table = BeautifulSoup(wiki, 'lxml')
    else:
        url = f'https://ru.wikipedia.org/wiki/{page_name}'
        r = get_url(url)
        wiki = r.text
        wiki = BeautifulSoup(wiki, 'lxml')
        table = wiki.find('tbody')  # для работы нам нужна лишь таблица
        write_data(file_name, data=minify(str(table)), is_raw=True)
    return table

In [16]:
def fix_bad_cities():
    bad_cities = get_bad_cities()
    print(f'Количество постов наблюдения с проблемными населёнными пунктами: {len(bad_cities)} пунктах')
    for city in bad_cities:
        try:
            def get_district(page):
                wiki = get_wiki_page(page)
                district = wiki.find('a', string="Муниципальный район")
                if not district:
                    if 'wiki_page' in city.keys() and page != city['wiki_page']:
                        #print(f"? пробуем {city['clean_name']} открыть как {city['wiki_page']}")
                        return get_district(city['wiki_page'])
                return district
            
            city_name = city['clean_name']
            district = get_district(city_name)
            if not district:
                print('- https://ru.wikipedia.org/wiki/' + city_name.replace(' ', '_'))
                continue

            district = district.parent.parent.find('td').find('a')['title']  # поиск строки с названием района
            print(f'+ {city_name} ({district})', end='')
            wiki = get_wiki_page(district)
            admin_town = wiki.find('th', string="Адм. центр")
            if not admin_town:
                print(f'\nАдминистративный центр для {city_name} ({district}) не обнаружен')
                continue
            admin_town = admin_town.parent.find('a').text
            admin_town = admin_town.replace('ё', 'е')  # в БД gismeteo нет букв Ё
            print(f' ==> {admin_town}', end='')
            admin_town_id = next((city_id for city_id, name in cities.items() if name == admin_town))
            admin_town_id = int(admin_town_id)
            print(f' ==> {int(admin_town_id)}', end='')
            for uid, item in posts.items():
                if item['clean_name'] == city_name and item['gismeteo_id'] != admin_town_id \
                    and ('fallback_id' not in item.keys() or item['fallback_id'] != admin_town_id):
                    print(f' ==> исправлено! ', end='')
                    if item['gismeteo_id'] == -1:
                        posts[uid].update({'gismeteo_id': admin_town_id})
                        print(f'Сделана замена')
                    else:
                        posts[uid].update({'fallback_id': admin_town_id})
                        print(f'Сделана подмена')
                    break
            
        except requests.exceptions.HTTPError:
            # Википедия даёт 404 статус на отсутствующие статьи, игнорируем
            print(f'Статья {city} не найдена')
            print('- https://ru.wikipedia.org/wiki/' + city)
            continue

fix_bad_cities()
print(f'Число проблемных населённых пунктов после исправления: {len(get_bad_cities())}')
posts

Количество постов наблюдения с проблемными населёнными пунктами: 23 пунктах
+ Чемдальск (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Усть-Камо (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана замена
- https://ru.wikipedia.org/wiki/Кузьмовка
+ Стрелка-Чуня (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Муторай (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.org/wiki/Вельмо
- https://ru.wikipedia.org/wiki/Тея
- https://ru.wikipedia.org/wiki/Бурный
+ Суломай (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.org/wiki/Светлана
- https://ru.wikipedia.org/wiki/Вельмо
+ Верхнекарелина (Киренский район) ==> Киренск ==> 4743 ==> исправлено! Сделана замена
- https://ru.wikipedia.org/wiki/Подволошино
- https://ru.wikipedia.org/wiki/Преображенка
+ Кислокан (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback_id': 4015},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': 4015},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка',
  'gismeteo_id': 158143,
  'fallback_id': -1},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня',
  'clean_name': 'Стрелка-Чуня',
  'gismeteo_id': 158151,
  'fallback_id': 4015},
 '09393': {'name': 'р.Чуня - пос.Муторай',
  'clean_name': 'Муторай',
  'gismeteo_id': 158146,
  'fallback_id': 4015},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е',
  'clean_name': 'Вельмо',
  'gismeteo_id': 157875,
  'fallback_id': -1},
 '09397': {'name':

Некоторые проблемные страницы открывают список страниц с похожими названиями.

![Несколько вариантов страниц в Википедии](images/wiki_variants.png)

В таких случаях необходимо вручную задать верную страницу на Википедии.

In [17]:
for uid, item in posts.items():
    new_name = item['clean_name']
    # Бакланиха  - ближайший к реке Большой порог населённый пункт
    new_name = new_name.replace('Большой Порог', 'Бакланиха_(Красноярский_край)')
    new_name = new_name.replace('Бурный', 'Бурный_(Эвенкийский_район)')
    new_name = new_name.replace('Вельмо', 'Вельмо_(посёлок)')
    new_name = new_name.replace('Кузьмовка', 'Кузьмовка_(Красноярский_край)')
    new_name = new_name.replace('Подволошино', 'Подволошино_(Иркутская_область)')
    new_name = new_name.replace('Преображенка', 'Преображенка_(Катангский район)')
    new_name = new_name.replace('Светлана', 'Вельмо_(посёлок)')
    new_name = new_name.replace('Тембенчи', 'Тура_(Красноярский_край)')
    new_name = new_name.replace('Тея', 'Тея_(Северо-Енисейский_район)')
    new_name = new_name.replace('Учами', 'Учами_(Полигусовский_сельсовет)')
    new_name = new_name.replace('Ика', 'Ика_(село)')
    new_name = new_name.replace('Непа', 'Непа_(село)')
    if item['clean_name'] != new_name:
        print(f"{item['clean_name']} теперь {new_name}")
        posts[uid].update({'wiki_page': new_name})

Кузьмовка теперь Кузьмовка_(Красноярский_край)
Вельмо теперь Вельмо_(посёлок)
Тея теперь Тея_(Северо-Енисейский_район)
Бурный теперь Бурный_(Эвенкийский_район)
Светлана теперь Вельмо_(посёлок)
Вельмо теперь Вельмо_(посёлок)
Подволошино теперь Подволошино_(Иркутская_область)
Преображенка теперь Преображенка_(Катангский район)
Учами теперь Учами_(Полигусовский_сельсовет)
Большой Порог теперь Бакланиха_(Красноярский_край)
Ика теперь Ика_(село)
Тембенчи теперь Тура_(Красноярский_край)
Большой Порог теперь Бакланиха_(Красноярский_край)
Непа теперь Непа_(село)


Повторно найдём замену для проблемных городов:

In [18]:
fix_bad_cities()
print(f'Число проблемных населённых пунктов после исправления: {len(get_bad_cities())}')

Количество постов наблюдения с проблемными населёнными пунктами: 14 пунктах
+ Кузьмовка (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Вельмо (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Тея (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Бурный (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Светлана (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана замена
+ Вельмо (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Подволошино (Катангский район Иркутской области) ==> Ербогачен ==> 4031 ==> исправлено! Сделана подмена
+ Преображенка (Катангский район) ==> Ербогачен ==> 4031 ==> исправлено! Сделана подмена
+ Учами (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Большой Порог (Туруханский район) ==> Туруханск ==> 3975 ==> исправлено! Сделана замена
+ Ика (Кат

Теперь для всех постов наблюдения можно получить полные метео-данные за указанный период.

Сохраним словарь с данными о пунктах наблюдений.

In [19]:
write_data(DATA_POSTS_FULL_RAW, data=posts, is_raw=True)
posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback_id': 4015},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': 4015},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка',
  'gismeteo_id': 158143,
  'fallback_id': 4015,
  'wiki_page': 'Кузьмовка_(Красноярский_край)'},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня',
  'clean_name': 'Стрелка-Чуня',
  'gismeteo_id': 158151,
  'fallback_id': 4015},
 '09393': {'name': 'р.Чуня - пос.Муторай',
  'clean_name': 'Муторай',
  'gismeteo_id': 158146,
  'fallback_id': 4015},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е',
  'clean_name': 'Вельмо',
  'gismeteo_id'

## Получение и сохранение метео-данных

Напишем функцию, которая запрашивает и обрабатывает html-страницу с метео-данными:

In [20]:
'''            url: https://www.gismeteo.ru/diary/4036/2008/1/
columns: [0]   [1]   [2]   [3]     [4]    [5]   [6]   [7]   [8]    [9]     [10]
row [0]      |             день               |            вечер               |
row [1] число|темп.|давл.|облач.|явления|ветер|темп.|давл.|облач.|явления|ветер|
       ------------------------------------------------------------------------|
row [2]   1  | -15 | 732 | img  |  img  |З 2мс| -14 | 733 | img  |  img  |З 2мс|
      -------------------------------------------------------------------------|
На некоторые дни нет метео-данных, вместо них - прочерки
Пример - 19 числа на https://www.gismeteo.ru/diary/4015/2010/12/
''' 
def process_row(row):
    cells = row.find_all('td')
    
    # day, temperature, pressure
    def get_cell_int(cell_id):
        if cell_id > 10:
            return None
        val = cells[cell_id].text
        if val == '−' or len(val) == 0:
            return get_cell_int(cell_id + 5)  # если за день нет информации, то берём её за вечер
        #print(val)
        return int(val)
        
    day = get_cell_int(0)
    temperature = get_cell_int(1)
    pressure = get_cell_int(2)
    
    # cloud, weather
    def get_cell_img(cell_id):
        if cell_id > 10:
            return None
        imgs = cells[cell_id].find_all('img')
        if len(imgs) > 0:
            if len(imgs) > 2:  # обычно в ячейке находятся 2 картинки - цветная и ч/б
                raise Exception(f"больше, чем 2 картинки")
            img = imgs[0]['src']  # //st6.gismeteo.ru/static/diary/img/dull.png
            if img.endswith('still.gif'):
                return get_cell_img(cell_id + 5)
            return re.search(r"diary/img/(\w+).png", img).group(1)
        return None
    
    cloud = get_cell_img(3)
    weather = get_cell_img(4)
    
    # wind info
    # Чаще всего текст в ячейке с погодой представлен как
    # направление_ветра скорость м/с
    # Штиль означается одной буквой Ш
    # в редких случаях пишется лишь скорость, пример - 27 число https://www.gismeteo.ru/diary/158155/2015/9/
    def get_wind_info(cell_id):
        if cells[cell_id].text == 'Ш':  # штиль
            return 'Ш', 0
        elif len(cells[cell_id].text) == 0:
            return get_wind_info(cell_id + 5)
        else:
            wind_info = re.search(r"(\w+) (\d+)", cells[cell_id].text)
            if wind_info:
                return wind_info.group(1), int(wind_info.group(2))
            wind_info = re.search(r"(\d+)", cells[cell_id].text)
            return 'Ш', int(wind_info.group(1))
            
    wind_dir, wind_spd = get_wind_info(5)

    # в реальных долгосрочных прогнозах погоды на весь день даётся 1 прогноз, без разделения по часам или времени суток,
    # поэтому не будем учитывать вечернюю погоду
    
    return [day, temperature, pressure, cloud, weather, wind_dir, wind_spd]
    #return {'day': day, 'temperature': temperature, 'pressure': pressure, 'cloud': cloud, 
    #        'weather': weather, 'day_wind_spd': day_wind_spd, 'day_wind_dir': day_wind_dir}
    #print(type(day), type(temperature), type(pressure), type(cloud), type(weather), type(day_wind_spd), type(day_wind_dir))

table = get_gismeteo_table(4036, 2008, 1)
data_rows = table.find_all('tr')
data_rows = data_rows[2:]  # убрать шапку таблицы
result = []
for row in data_rows:
    dict_item = process_row(row)
    result.append(dict_item)
print(dict_item)

[31, -17, 745, 'sunc', None, 'ЮЗ', 2]


Теперь сделаем функцию, которая будет сохранять помесячно метео-данные в формате csv:

In [21]:
from datetime import datetime
import csv
import os
from calendar import monthrange

START_YEAR = 2008
END_YEAR = 2018
# id_поста,дата,температура,давление,облачность,осадки,скорость_ветра,направление_ветра
header = ['uid', 'date', 'temperature', 'pressure', 'cloud', 'weather', 'wind_dir', 'wind_spd']

# Для данных параметров все необходимые страницы gismeteo будут загружаться минимум 4 часа

# Для ускорения процесса парсинга сайта можно использовать прокси и многопоточность через
# встроенную библиотеку multiprocessing, но для этичности проекта и избежания излишней 
# нагрузки на сайт запросы будут производиться в однопоточном режиме

result = []
if is_data_exists(DATA_WEATHER, is_raw=True):
    print(f"Датасет {DATA_WEATHER} уже сформирован.")
else:
    for uid, post in posts.items():
        for year in range(START_YEAR, END_YEAR + 1):        
            print(f"{post['name']} - год {year}… ", end='')
            for month in range(1, 13):
                print(f"{month}…", end='')

                def get_data_rows(city_id):
                    table = get_gismeteo_table(city_id, year, month)
                    if not table:
                        return None

                    data_rows = table.find_all('tr')
                    return data_rows[2:]  # убрать шапку таблицы

                # В приоритете используются метео-данными из основного источника, если их
                # Примеры неполных метеоданных:
                # с 18 + пропуски https://www.gismeteo.ru/diary/158155/2015/3/
                # до 19 числа https://www.gismeteo.ru/diary/4015/2015/9/
                # с 29 по 31 https://www.gismeteo.ru/diary/4015/2015/10/
                main_data = get_data_rows(post['gismeteo_id'])
                fallback_dict = None

                if not main_data or (len(main_data) < monthrange(year, month)[1] \
                                     and 'fallback_id' in post.keys()):                 
                    fallback_data = get_data_rows(post['fallback_id'])
                    if not fallback_data:
                        raise Exception(f"something goes wrong with {post['gismeteo_id']} \
                                        and {post['fallback_id']} {year} {month}, halt")
                    if main_data:
                        fallback_data = [process_row(row) for row in fallback_data]
                        fallback_dict = {day: (temperature, pressure, cloud, weather, wind_dir, wind_spd)
                                        for day, temperature, pressure, cloud, weather, wind_dir, wind_spd in fallback_data}
                    else:
                        main_data = fallback_data

                day = 1 
                for row in main_data:
                    try:
                        data = process_row(row)
                    except Exception:
                        err_str = f"check https://www.gismeteo.ru/diary/{post['gismeteo_id']}/{year}/{month}/"
                        if 'fallback_id' in post.keys():
                            err_str += f"\nand https://www.gismeteo.ru/diary/{post['fallback_id']}/{year}/{month}/"
                        raise Exception(err_str)

                    row_day = data[0]
                    while day != row_day:  # есть пропуски в днях в основном источнике метео-данных
                        if fallback_dict:  # заполняем пропуски из второстепенного источника
                            #print(f'\nuse fallback to fill {day} day')
                            if day in fallback_dict:
                                date_string = f'{year}-{month:02d}-{day:02d}'
                                result.append([uid, date_string] + list(fallback_dict[day]))
                                #print(fallback_dict[day])
                                day += 1
                                continue
                        # в обоих источниках есть пропуски данных - заполняем день пустыми данными
                        date_string = format_data(year, month, day)
                        result.append([uid, date_string] + [None for x in range(len(header) - 2)])
                        #print(f'No data for {date_string}')
                        day += 1

                    date_string = f'{year}-{month:02d}-{row_day:02d}'
                    result.append([uid, date_string] + data[1:])
                    day += 1
            print()  # перенос строки
    write_csv(DATA_WEATHER, header, result, is_raw=True)

р.Подкаменная Тунгуска - пос.Чемдальск - год 2008… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2009… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2010… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2011… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2012… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2013… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2014… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2015… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2016… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2017… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Чемдальск - год 2018… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - с.Ванавара - год 2008… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - с.Ванавара - год 2009… 1…2…3…4

р.Подкаменная Тунгуска - пос.Суломай - год 2013… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Суломай - год 2014… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Суломай - год 2015… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Суломай - год 2016… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Суломай - год 2017… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Подкаменная Тунгуска - пос.Суломай - год 2018… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2008… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2009… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2010… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2011… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2012… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2013… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2014… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - год 2015… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Вельмо - пос.Светлана - 

р.Нижняя Тунгуска - факт.Большой Порог - год 2016… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Нижняя Тунгуска - факт.Большой Порог - год 2017… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Нижняя Тунгуска - факт.Большой Порог - год 2018… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2008… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2009… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2010… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2011… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2012… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2013… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2014… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2015… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2016… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2017… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - с.Токма - год 2018… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - д.Ика - год 2008… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - д.Ика - год 2009… 1…2…3…4…5…6…7…8…9…10…11…12…
р.Непа - д.Ика - год 2010…